# ATEK Data Proprocessing Example (EFM model)

### 1. Load the example adt data

In [6]:
import os

from atek.data_preprocess.sample_builders.efm_sample_builder import EfmSampleBuilder
from atek.data_preprocess.wds_writer import AtekWdsWriter

from omegaconf import OmegaConf

import logging
from logging import StreamHandler

handler = StreamHandler()
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

logger = logging.getLogger()
logger.addHandler(handler)

example_adt_data_dir = "/home/louy/Calibration_data_link/Atek/2024_05_07_EfmDataTest/adt_data_example/Apartment_release_clean_seq134/1WM103600M1292_optitrack_release_clean_seq134"
config_path = os.path.join("/home/louy/fbsource/fbcode/surreal/data_services/atek/atek/configs/efm_preprocess_base.yaml")

conf = OmegaConf.load(config_path)

### 2. Sample builder example

In [ ]:
sample_builder = EfmSampleBuilder(
    conf=conf.processors,
    vrs_file=os.path.join(example_adt_data_dir, "video.vrs"),
    mps_files={
        "mps_closedloop_traj_file": os.path.join(
            example_adt_data_dir, "aria_trajectory.csv"),
        "mps_semidense_points_file": os.path.join(example_adt_data_dir, "mps", "slam", "semidense_points.csv.gz"),
        "mps_semidense_observations_file": os.path.join(example_adt_data_dir, "mps", "slam", "semidense_observations.csv.gz"),
    },
    gt_files={
        "obb3_file": os.path.join(example_adt_data_dir, "3d_bounding_box.csv"),
        "obb3_traj_file": os.path.join(
            example_adt_data_dir, "scene_objects.csv"
        ),
        "instance_json_file": os.path.join(
            example_adt_data_dir, "instances.json"
        ),
    },
    depth_vrs_file = os.path.join(example_adt_data_dir, "depth_images.vrs")
)



In [ ]:
# Print the frame dataclass keys
sample = sample_builder.get_sample_by_timestamps_ns([14601564806000])
from dataclasses import asdict
print(asdict(sample).keys())

### 2. Write wds files

In [ ]:
from atek.data_preprocess.wds_writer import (
    AtekWdsWriter,
)
from atek.data_preprocess.subsampling_lib.temporal_subsampler import CameraTemporalSubsampler

subsampler = CameraTemporalSubsampler(vrs_file = os.path.join(example_adt_data_dir, "video.vrs"), conf = conf.camera_temporal_subsampler)

conf.wds_writer.prefix_string = "test"
atek_wds_writer = AtekWdsWriter(output_path = "/home/louy/Calibration_data_link/Atek/2024_05_07_EfmDataTest/wds_output/test_1", conf = conf.wds_writer)

for i in range(subsampler.get_total_num_samples()):
    timestamps_ns = subsampler.get_timestamps_by_sample_index(i)

    sample = sample_builder.get_sample_by_timestamps_ns(timestamps_ns)
    if sample is not None:
        atek_wds_writer.add_sample(data_sample = sample)

atek_wds_writer.close()
